#  Roundoff Error and Stability

> ***The purpose of computing is insight, not numbers (Hamming).***

As we discussed in class, roundoff error is a fact of life when computing numerically. As computations proceed these roundoff errors can accumulate and cause many difficulties. As long as the roundoff errors grow linearly with the number of steps, then we can expect that our final results should be reasonable (barring an ill-conditioned problem).

This problem demonstrates some of the pitfalls when trying to compute various integrals and some possible alternative formulations to avoid them.


## Problem 5 
***Suppose we are asked to compute $$y_n = \int_0^1 \frac{x^n}{x+10}dx, \ n =1,2, \ldots 30 \qquad (1) $$***

a) ***Show that*** $$
y_n + 10 y_{n-1} = \frac{1}{n}
$$

***Solution:***

Let's first note that $$
\begin{aligned}
y_n + 10 y_{n-1} &= \int_0^1 \frac{x^n}{x+10}dx + 10 \int_0^1 \frac{x^{n-1}}{x+10}dx \qquad \mbox{by definition} \\
&= \int_0^1 \frac{x^n + 10 x^{n-1}}{x+10}dx \qquad \qquad \mbox{combine terms} \\
&= \int_0^1 \frac{x^{n-1}(x + 10) }{x+10}dx, \qquad \mbox{factor out  } x^{n-1}\\
&= \int_0^1 x^{n-1}dx = \frac{x^n}{n} \Bigg |_0^1 \\
y_n + 10 y_{n-1} &= \frac{1}{n}
\end{aligned}
$$

or solving for $y_n$: $$
y_n =  \frac{1}{n} - 10 y_{n-1} 
$$

b) ***Write a code to numerically compute a value for***  $y_0$.

***Solution:***

Note that for $n=0$ Equation (1) reduces to: $$
\begin{aligned}
y_0 &= \int_0^1 \frac{1}{x+10}dx = \ln (x + 10) \Bigg |_0^1 \\
&= \ln(11) - \ln(10) \\
&= 0.0953101798043
\end{aligned}
$$

c) ***Using (a) and (b) propose an algorithm and write a code that computes $y_n, n = 1, 2, \ldots, 30$. You may not use any other software except for what you write yourself.***

***Solution:***

## Algorithm:

This suggests the following procedure:

1.  Evaluate $y_0 = \ln(11) - \ln(10)$

2.  For $n=1, 2, \ldots 30$

    -   $y_n = \frac{1}{n} - 10 y_{n-1}$


***Important:***


Before we even start, you may want to note that all of the integrals must be positive. In fact, we can show that $$
0 < y_n < 1, \quad \forall n=1, 2, \ldots, 30$$
so we have a sense of what reasonable values should be.

Let's see how we might be able to implement the above algorithm in python

In [10]:
# Setup numpy environment
import numpy as np
from scipy import io, integrate, linalg, signal
from scipy.sparse.linalg import cg, eigs

# Initialize
naxis = np.arange(31)
y = np.zeros(31)
y[0] = np.log(11) - np.log(10)

# Compute the rest of the integrals
for n in range(1,31):
    y[n] = 1/n - 10*y[n-1]

# View the results
y

array([ 9.53101798e-02,  4.68982020e-02,  3.10179804e-02,  2.31535290e-02,
        1.84647099e-02,  1.53529009e-02,  1.31376581e-02,  1.14805618e-02,
        1.01943816e-02,  9.16729515e-03,  8.32704853e-03,  7.63860560e-03,
        6.94727731e-03,  7.45030378e-03, -3.07446639e-03,  9.74113306e-02,
       -9.11613306e-01,  9.17495659e+00, -9.16940103e+01,  9.16992735e+02,
       -9.16987735e+03,  9.16988211e+04, -9.16988166e+05,  9.16988170e+06,
       -9.16988169e+07,  9.16988170e+08, -9.16988170e+09,  9.16988170e+10,
       -9.16988170e+11,  9.16988170e+12, -9.16988170e+13])

Viewing the results by typing the variable is the easiest, but sometimes the output is hard to read and therefore hard to interpret. Let's see how we can make the output look a bit clearer using the python package pandas

In [11]:
import pandas as pd 

# Create a data frame using pandas

table = np.column_stack((naxis,y))
table = pd.DataFrame(table, columns=['n', 'y_n'])
table

,n,y_n
0,0.0,9.531018e-02
1,1.0,4.689820e-02
2,2.0,3.101798e-02
3,3.0,2.315353e-02
4,4.0,1.846471e-02
5,5.0,1.535290e-02
6,6.0,1.313766e-02
7,7.0,1.148056e-02
8,8.0,1.019438e-02
9,9.0,9.167295e-03


## Observations

List any insights you have after viewing your preliminary results.

1. The values of the integrals appear correct for the first few numbers as compared with the table given
2. Entry #14 is negative, which should not have been possible. Some of the other later entries are also negative.
3. The entries start to grow at about entry #15 and are soon greater than 1, which also should not be possible.
4. All of the latter entries grow (approximately) by a factor of 10.

(d) ***The true values for the integrals are given in Table 1 of the homework sheet.  For each of the $y_n$, compute the actual and relative errors (where applicable). Analyze and explain your results in terms of what we discussed in class on the stability of algorithms.***

Note: You could input the values from the table, but for the sake of convenience we'll provide you the code that produced the numbers. Let's compute the actual values of the integrals and check the error. We will introduce methods for numerically computing integrals later in the course. For now, let's just say that the function np.trapz, will numericall evaluate the integral above.

In [12]:
x = np.linspace(0,1,num=50)
yint = np.zeros(31)

# Compute the integrals using the Trapezoidal rule
for n in range(0,30):
    yn = x**n / (x + 10)
    yint[n] = np.trapz(yn,x)

yint

array([0.09531024, 0.0468976 , 0.031024  , 0.02316271, 0.01847704,
       0.01536839, 0.0131563 , 0.01150236, 0.01021934, 0.0091953 ,
       0.00835922, 0.00766384, 0.00707652, 0.00657401, 0.00613926,
       0.0057595 , 0.00542499, 0.00512816, 0.00486303, 0.00462483,
       0.00440971, 0.00421449, 0.00403658, 0.00387381, 0.00372434,
       0.00358665, 0.00345942, 0.00334152, 0.00323199, 0.00312998,
       0.        ])

## Error Analysis

To fully understand what is going on, we need to take a look at the error between the algorithm's output and the "true" value computed by the np.trapz function (or the given table).

What does the data show?

In [13]:
# Compute the absolute error

abserr = abs(yint - y)
table = np.column_stack((naxis,abserr))
table = pd.DataFrame(table, columns=['n', 'abserr'])
table

,n,abserr
0,0.0,6.023656e-08
1,1.0,6.023656e-07
2,2.0,6.023656e-06
3,3.0,9.178958e-06
4,4.0,1.233371e-05
5,5.0,1.548820e-05
6,6.0,1.864230e-05
7,7.0,2.179464e-05
8,8.0,2.495730e-05
9,9.0,2.800810e-05


## Error Analysis (part 2)

If we take a look at the error, you'll see that we have an accurate approximation at the start - on the order of $10^{-8}.$ But the error quickly builds up until by the 16th value the absolute error is $\approx 1$. 

Since the formula for computing each integral involves a subtraction, 

$$
y_n  = \frac{1}{n} - 10 y_{n-1}
$$

it makes sense to suspect there might be some cancellation error. If we take a look at the two quantities involved we might be able to determine what is happening. (I'll leave that up to you.)

***Another explanation is that at each iteration the value of the integral $y_{n-1}$ has some roundoff error.  When we multiply that quantity by 10, the error is also magnified by a factor of 10.*** 

As a result, the next computed integral's error is off by another factor of 10.

Recall from class lecture:
- Let $E_n$ be the error at the $nth$ step of some computation. If the error $E_n \approx C\cdot n E_0,$ where $C$ is a constant and $E_0$ is the original error, then the growth of error is said to be ***linear***.-   If the error $E_n \approx C^n E_0,$ where $C$ is a constant and $E_0$ is the original error, then the growth of error is said to be ***exponential***.


This is a clear signal of an **unstable algorithm**

In [14]:
# Compute the relative error

abserr = abs(yint - y)/abs(yint)
table = np.column_stack((naxis,abserr))
table = pd.DataFrame(table, columns=['n', 'abserr'])
table

/var/folders/ks/_91630850zn018jzskzpdw1c0000gp/T/ipykernel_7615/1550262451.py:3: RuntimeWarning: divide by zero encountered in divide
  abserr = abs(yint - y)/abs(yint)


,n,abserr
0,0.0,6.320052e-07
1,1.0,1.284427e-05
2,2.0,1.941612e-04
3,3.0,3.962817e-04
4,4.0,6.675151e-04
5,5.0,1.007796e-03
6,6.0,1.416987e-03
7,7.0,1.894798e-03
8,8.0,2.442164e-03
9,9.0,3.045914e-03


Here we once again notice a similar pattern in the error in that it appears to be growing by a factor of 10 after about the 11th iteration.

Note also that we get an "inf" in the last entry. 

If we look back at the computed solutions from the trapezoid routine (or the given table), we note that the last entry is exactly 0, hence the floating point exception. And why we need to be careful when computing the relative error.

e) ***Modify your code so that it can compute more accurate values for the integral (without resorting to numerical integration or any other software). Justify your algorithm, demonstrate why it is more accurate,  and provide all code.***

## Solution (e)

The key observation here is that the recurrence relation for the integrals is valid whether we go forward or backward.  

The only hitch is that while we know the value of $y_0$, it isn't as clear what we should use for the last value of $y_n$ nor which $n$ to use.

But noticing that the value of computed by the trapezoid rule was $y_{30} = 0$, we can use that as a *guess* and then run the for loop in reverse. 

In [20]:
# Initialize
naxis = np.arange(31)
yrev = np.zeros(31)
yrev[30] = 0.0

# Compute the rest of the integrals
for n in range(30, 0, -1):
    yrev[n-1] = (1/n - yrev[n])/10

yrev

array([0.09531018, 0.0468982 , 0.03101798, 0.02315353, 0.01846471,
       0.0153529 , 0.01313766, 0.01148056, 0.01019439, 0.0091672 ,
       0.00832797, 0.00762944, 0.00703898, 0.00653332, 0.00609542,
       0.00571251, 0.00537486, 0.00507489, 0.00480663, 0.0045653 ,
       0.00434704, 0.00414869, 0.00396765, 0.00380175, 0.00364916,
       0.00350838, 0.00337771, 0.00325993, 0.00311494, 0.00333333,
       0.        ])

Let's compute the absolute error again using the revised algorithm

In [21]:
# Compute the error - note the use of absolute rather than relative 
# since the last value of yint = 0.0

abserr2 = abs(yint - yrev)
table = np.column_stack((naxis,abserr2))
table = pd.DataFrame(table, columns=['n', 'abserr2'])
table

,n,abserr2
0,0.0,6.023656e-08
1,1.0,6.023656e-07
2,2.0,6.023656e-06
3,3.0,9.178958e-06
4,4.0,1.233370e-05
5,5.0,1.548821e-05
6,6.0,1.864221e-05
7,7.0,2.179556e-05
8,8.0,2.494813e-05
9,9.0,2.809980e-05
